<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Decorator-Basics" data-toc-modified-id="Decorator-Basics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Decorator Basics</a></span><ul class="toc-item"><li><span><a href="#Syntax-variants" data-toc-modified-id="Syntax-variants-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Syntax variants</a></span></li><li><span><a href="#Support-Input-Arguments-for-Decorated-Function" data-toc-modified-id="Support-Input-Arguments-for-Decorated-Function-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Support Input Arguments for Decorated Function</a></span></li><li><span><a href="#Handle-Returns-of-The-Decorated-Functions" data-toc-modified-id="Handle-Returns-of-The-Decorated-Functions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Handle Returns of The Decorated Functions</a></span></li><li><span><a href="#Wrap-the-Decorated-Function-to-Preserve-Metadata" data-toc-modified-id="Wrap-the-Decorated-Function-to-Preserve-Metadata-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Wrap the Decorated Function to Preserve Metadata</a></span></li><li><span><a href="#Define-Decorators-With-Arguments" data-toc-modified-id="Define-Decorators-With-Arguments-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Define Decorators With Arguments</a></span></li><li><span><a href="#Class-based-Decorators" data-toc-modified-id="Class-based-Decorators-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Class-based Decorators</a></span></li></ul></li><li><span><a href="#Logging-Decorator" data-toc-modified-id="Logging-Decorator-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Logging Decorator</a></span></li><li><span><a href="#Retry-Decorator" data-toc-modified-id="Retry-Decorator-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Retry Decorator</a></span></li></ul></div>

In [8]:
import sys
import time

In [9]:
print(sys.executable)
print(sys.version)

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]


## Decorator Basics

### Syntax variants

In basic terms, a decorator is a callable that takes a second callable as input and returns another (third) callable, the so called wrapper / closure.

The wrapper runs the original input function, and modifies it's result. Decorators modify the behavior of a callable through a wrapper so you don’t have to permanently modify the original. It's behavior changes only when decorated.

Step by step:
- Firstly, define the actual decorator. It accepts a single argument, which is the function we are trying to decorate. It will return the "wrapper".
- Inside, we define a wrapper function that _is returned_ and used in place of the original decorated function.

In [10]:
# Basic time logging decorator, for a function that does not take any arguments

def logging_time(func):
    """Decorator that logs time."""
    def logger_wrapper():
        """Function that logs time."""
        start = time.time()
        func()
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")
    return logger_wrapper

In [13]:
@logging_time
def calculate_sum():
    return sum(range(1000000))

In [14]:
calculate_sum()

Calling calculate_sum: 0.04600


In [11]:
# ALTERNATIVE SYNTAX with manual decoration - you define the input function "stand-alone"

def calculate_sum():
    return sum(range(1000000))

calculate_sum = logging_time(calculate_sum)

In [12]:
calculate_sum()

Calling calculate_sum: 0.04703


**_"Note that using the @ syntax decorates the function immediately at definition time. This makes it difficult to access the undecorated original without brittle hacks. Therefore you might choose to decorate some functions manually in order to retain the ability to call the undecorated function as well."_**

### Support Input Arguments for Decorated Function

There is a problem with the code snippet above: It assumes that the decorated functions don’t require any input arguments.
To address this issue, we should consider using * args and ** kwargs with the _inner_ decorator definition.     


In [6]:
# Revised version capable of handling input arguments

def logging_time(func):
    """Decorator that logs time."""
    def logger(*args, **kwargs):
        """Function that logs time."""
        start = time.time()
        func(*args, **kwargs)
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")

    return logger

In [7]:
@logging_time
def calculate_sum_n(n):
    return sum(range(n))

@logging_time
def say_hi(whom, greeting="Hello"):
    """Greet someone"""
    print(f"{greeting}, {whom}!")

In [8]:
calculate_sum_n(100000)

Calling calculate_sum_n: 0.00400


In [9]:
say_hi("raph", "hi")

hi, raph!
Calling say_hi: 0.00000


### Handle Returns of The Decorated Functions

You can see we store the returned value in result on line 4. But before returning it, we have to finish timing the function. This is an example of behaviour that would not be possible without decorators.

In [10]:
# Slightly different logging example

def timer_decorator(func):
    def timer_wrapper(*args, **kwargs):
        import datetime as dt
        before = dt.datetime.now()                     
        result = func(*args,**kwargs)                
        after = dt.datetime.now()                      
        print("Elapsed Time = {after-before}")
        return result
    
    return timer_wrapper

In [11]:
@timer_decorator
def sum_function(x, y):
    print(x + y)
    return x + y

In [12]:
result = sum_function(2, 5)
print(f"\nResult: {result}")

7
Elapsed Time = {after-before}

Result: 7


### Wrap the Decorated Function to Preserve Metadata

The decoration will by default mess up the metadata of the decorated function, such as docstrings. To solve this problem, we can use another decorator function (wraps) that is shipped in the standard Python library, as shown below.

**_"As a best practice, I’d recommend that you use functools.wraps in all of the decorators you write yourself. It doesn’t take much time and it will save you (and others) debugging headaches down the road. (Dan Bader, Python Tricks, p. 84)"_**

In [13]:
# The decorated function now has the wrong docstrings.
print(say_hi.__doc__)

Function that logs time.


In [14]:
# Revised decorator using the imported wraps() decorator

import functools

def logging_time(func):
    """Decorator that logs time."""
    @functools.wraps(func)
    def logger(*args, **kwargs):
        """Function that logs time."""
        start = time.time()
        func(*args, **kwargs)
        print(f"Calling {func.__name__}: {time.time() - start:.5f}")

    return logger

In [15]:
@logging_time
def say_hi(whom, greeting="Hello"):
    "Greet someone."
    print(f"{greeting}, {whom}!")

In [16]:
# The decorated function now has the correct docstrings.
print(say_hi.__doc__)

Greet someone.


### Define Decorators With Arguments

The reason for adding another layer to get the decorator to accept arguments is that the decoration process is chaining the function call. Calling logging_time ("ms") will allow us to get the logger function, which has exactly the same function signature as the decorator function that we defined earlier.

Please note that the current definition of the decorators _requires_ that we specify the unit for the decoration. If you want to make your arguments optional, it needs extra work. (see [here](https://medium.com/better-programming/how-to-write-python-decorators-that-take-parameters-b5a07d7fe393) for more info).

In [17]:
# Refined decorator to display the time in the unit that’s specified by the user
# (either in milliseconds or seconds).

def logging_time(unit):
    """Decorator that logs time."""
    def logger(func):
        @wraps(func)
        def inner_logger(*args, **kwargs):
            """Function that logs time."""
            start = time.time()
            func(*args, **kwargs)
            scaling = 1000 if unit == "ms" else 1
            print(f"Calling {func.__name__}: {(time.time() - start) * scaling:.5f} {unit}")

        return inner_logger

    return logger

In [18]:
@logging_time("ms")
def calculate_sum_ms(n):
    """Calculate sum of 0 to n-1"""
    return sum(range(n))

@logging_time("s")
def calculate_sum_s(n):
    """Calculate sum of 0 to n-1"""
    return sum(range(n))

In [19]:
calculate_sum_ms(10000000)

Calling calculate_sum_ms: 528.99837 ms


In [20]:
calculate_sum_s(10000000)

Calling calculate_sum_s: 0.42451 s


### Class-based Decorators

It is possible to decorate using classes instead of functions. The only difference is the syntax, so do what you are more comfortable with.

In [21]:
# Example of a logging decorator class

class Logging: 
  
    def __init__(self, function): 
        self.function = function 
  
    def __call__(self, *args, **kwargs):
      print(f'Before {self.function.__name__}')
      self.function(*args, **kwargs)
      print(f'After {self.function.__name__}')

In [22]:
@Logging
def sum(x, y):
  print(x + y)

In [23]:
sum(3, 7)

Before sum
10
After sum


In [24]:
# Example of a repeater decorator class (accecpting an argument)

class Repeat:
    def __init__(self, n):
        self.n = n

    def __call__(self, func):
        def repeater(*args, **kwargs):
            for _ in range(self.n):
                func(*args, **kwargs)

        return repeater
    

In [25]:
@Repeat(n=2)
def morning_greet(person):
    print(f"Good Morning, {person}!")

In [26]:
morning_greet("raph")

Good Morning, raph!
Good Morning, raph!


---

## Logging Decorator

---

## Retry Decorator

Note: This example is more for practice / understanding. There are python packages like _tenacity_ or _retry_ that provide fully tested retry decorators with more functionality.

code mostly take from this [blogpost](https://towardsdatascience.com/are-you-using-python-with-apis-learn-how-to-use-a-retry-decorator-27b6734c3e6)

In [28]:
import logging
import random
import time
from functools import wraps
from typing import Optional, Tuple, Union

logger = logging.getLogger(__name__)

In [29]:
type(logger)

logging.Logger

In [30]:
def retry(exceptions: Union[Tuple, str] = Exception,
          total_tries: int = 4,
          initial_wait: float = 0.5,
          backoff_factor: Union[int, float] = 2, 
          logger: Optional[logging.Logger] = None
    ):
    """Calling the decorated function,  applying an exponential backoff.
    
    Args:
    - exceptions: Exeption(s) that trigger a retry, can be a tuple like: 
        (ConnectionAbortedError, ConnectionRefusedError, ConnectionResetError).
    - total_tries: Total tries before fail.
    - initial_wait: Time to first retry.
    - backoff_factor: Backoff multiplier (e.g. value of 2 will double the delay each retry).
    - logger: logger to be used, if none specified print to standard out.
    """
    def retry_wrapper(func):
        @wraps(func)
        def func_with_retries(*args, **kwargs):
            _tries, _delay = total_tries + 1, initial_wait
            while _tries > 1:
                try:
                    _log(f"{total_tries + 2 - _tries}. try: {logger}")
                    return func(*args, **kwargs)
                except exceptions as e:
                    _tries -= 1
                    print_args = args if args else "no args"
                    if _tries == 1:
                        msg = str(f"Function: {func.__name__}\n"
                                  f"Failed despite best efforts after {total_tries} tries.\n"
                                  f"args: {print_args}, kwargs: {kwargs}")
                        _log(msg, logger)
                        raise
                    msg = str(f"Function: {func.__name__}\n"
                              f"Exception: {e}\n"
                              f"Retrying in {_delay} seconds!, args: {print_args}, kwargs: {kwargs}\n")
                    _log(msg, logger)
                    time.sleep(_delay)
                    _delay *= backoff_factor

        return func_with_retries
    return retry_wrapper


def _log(msg, logger=None):
    if logger:
        logger.warning(msg)
    else:
        print(msg)

In [21]:
# Test it

@retry(Exception, total_tries=2, logger=None)
def test_func(*args, **kwargs):
    rnd = random.random()
    if rnd < .2:
        raise ConnectionAbortedError('Connection was aborted :(')
    elif rnd < .4:
        raise ConnectionRefusedError('Connection was refused :/')
    elif rnd < .8:
        raise ConnectionResetError('Guess the connection was reset')
    else:
        return 'Success!'


# if __name__ == '__main__':
    # retry_wrapper = retry((ConnectionAbortedError), tries=3, delay=.2, backoff=1, logger=logger)
    # wrapped_test_func = retry_wrapper(test_func)
    # print(wrapped_test_func('hi', 'bye', hi='ciao'))

In [22]:
test_func('hi', 'bye', hi='ciao')

1. try: None
Function: test_func
Exception: Connection was refused :/
Retrying in 0.5 seconds!, args: ('hi', 'bye'), kwargs: {'hi': 'ciao'}

2. try: None
Function: test_func
Failed despite best efforts after 2 tries.
args: ('hi', 'bye'), kwargs: {'hi': 'ciao'}


ConnectionAbortedError: Connection was aborted :(

---